<a href="https://colab.research.google.com/github/imy0ung/AI_Basic_Study/blob/main/Pytorch/Pytorch_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Optimization**
* Forwardpropagation을 통해 Loss을 계산하였다면, loss function을 기준으로 최적의 가중치를 SGD를 이용해 구해야함.
* 최적의 가중치를 찾는 과정에서 우리는 Hyperparmeter(수정할 수 있는 매겨변수)을 통해 SGD의 여러 방법을 설정해 줄 수 있음
* 업데이트 할 가중치는 Backpropation을 통해 Update됨

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [18]:
training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size = 64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module) :
    def __init__(self) :
      super().__init__()
      self.flatten = nn.Flatten()
      self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10)
    )
    def forward(self,x) :
      x = self.flatten(x)
      logits = self.linear_relu_stack(x)
      return logits

model = NeuralNetwork()

### **hyperparmeters**

* epoch : 데이터셋을 반복하는 횟수
* batch size : 신경망을 통해 전파될 데이터 sample
* learning rate(step-size) : 각 batch/epoch에서 모델의 매개변수를 조절하는 비율

In [4]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [5]:
# loss function
loss_fn = nn.CrossEntropyLoss()

In [6]:
# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [20]:
def train_loop(dataloader, model, loss_fn, optimizer) :
    # dataloader로 불러온 데이터의 길이를 셈
    size = len(dataloader.dataset)
    # model을 학습 모드로 설정
    model.train()

    for batch, (X,y) in enumerate(dataloader) : # X는 입력데이터 y는 label, batch는 index (shuffle을 안했으므로, 10개씩 나옴 )
      # forwardpropagation으로 loss을 구함
      pred = model(X)
      loss = loss_fn(pred, y)

      # backpropagation
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      if batch % 100 == 0 :
        loss, current = loss.item(), batch * batch_size + len(X)
        print(f"loss : {loss:>7f} [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn) :
  # model을 평가 모드로 설정합니다.
  model.eval()
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0

  with torch.no_grad() :
    for X, y in dataloader :
      pred = model(X)
      test_loss += loss_fn(pred,y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [21]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss : 2.305376 [   64/60000]
loss : 2.287003 [ 6464/60000]
loss : 2.269340 [12864/60000]
loss : 2.264924 [19264/60000]
loss : 2.250109 [25664/60000]
loss : 2.227501 [32064/60000]
loss : 2.231972 [38464/60000]
loss : 2.197610 [44864/60000]
loss : 2.190740 [51264/60000]
loss : 2.177950 [57664/60000]
Test Error: 
 Accuracy: 48.3%, Avg loss: 2.157796 

Epoch 2
-------------------------------
loss : 2.162119 [   64/60000]
loss : 2.150490 [ 6464/60000]
loss : 2.096777 [12864/60000]
loss : 2.120107 [19264/60000]
loss : 2.070513 [25664/60000]
loss : 2.018619 [32064/60000]
loss : 2.048111 [38464/60000]
loss : 1.968239 [44864/60000]
loss : 1.970630 [51264/60000]
loss : 1.924470 [57664/60000]
Test Error: 
 Accuracy: 51.7%, Avg loss: 1.899566 

Epoch 3
-------------------------------
loss : 1.932652 [   64/60000]
loss : 1.900500 [ 6464/60000]
loss : 1.785226 [12864/60000]
loss : 1.827657 [19264/60000]
loss : 1.724787 [25664/60000]
loss : 1.679764 [32064/600